<a href="https://colab.research.google.com/github/williamchi64/williamchi64/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E5%81%9A%E5%9C%96%E5%83%8F%E8%B3%87%E6%96%99%E5%88%86%E5%89%B2%E8%88%87training_inceptV3(%E4%B8%8A%E8%AA%B2%E6%9D%90%E6%96%99).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon May 10 15:18:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install gdown

In [ ]:
!wget -P /content/datasets https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
!tar -xf /content/datasets/wiki_crop.tar

--2021-05-10 15:18:07--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘/content/datasets/wiki_crop.tar.1’

wiki_crop.tar.1     100%[===================>] 773.73M  18.8MB/s    in 35s     

2021-05-10 15:18:43 (22.0 MB/s) - ‘/content/datasets/wiki_crop.tar.1’ saved [811315200/811315200]



In [ ]:
!wget --output-document=/content/datasets/wiki_fillgdr.csv drive.google.com/u/0/uc?id=1_LYKqaJ6lwFwT1xKZnjzV2mpkiGG56nF&export=download

--2021-05-10 15:34:40--  http://drive.google.com/u/0/uc?id=1_LYKqaJ6lwFwT1xKZnjzV2mpkiGG56nF
Resolving drive.google.com (drive.google.com)... 172.217.164.174, 2607:f8b0:4004:815::200e
Connecting to drive.google.com (drive.google.com)|172.217.164.174|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/u/0/uc?id=1_LYKqaJ6lwFwT1xKZnjzV2mpkiGG56nF [following]
--2021-05-10 15:34:40--  https://drive.google.com/u/0/uc?id=1_LYKqaJ6lwFwT1xKZnjzV2mpkiGG56nF
Connecting to drive.google.com (drive.google.com)|172.217.164.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hok7f8sm410vv8mqh9b5djkpd74m96ti/1620660825000/01254859074079537426/*/1_LYKqaJ6lwFwT1xKZnjzV2mpkiGG56nF [following]
--2021-05-10 15:34:41--  https://doc-0g-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hok

In [ ]:
!wget --output-document=/content/datasets/del.csv drive.google.com/u/0/uc?id=1lHtMfINNtwfpTMQ2zJny_AiyFpHP9puM&export=download

--2021-05-10 15:19:09--  http://drive.google.com/u/0/uc?id=1lHtMfINNtwfpTMQ2zJny_AiyFpHP9puM
Resolving drive.google.com (drive.google.com)... 172.217.164.174, 2607:f8b0:4004:815::200e
Connecting to drive.google.com (drive.google.com)|172.217.164.174|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/u/0/uc?id=1lHtMfINNtwfpTMQ2zJny_AiyFpHP9puM [following]
--2021-05-10 15:19:09--  https://drive.google.com/u/0/uc?id=1lHtMfINNtwfpTMQ2zJny_AiyFpHP9puM
Connecting to drive.google.com (drive.google.com)|172.217.164.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n69gd6tsq1vvt8tirlo4vcfrrrde70ce/1620659925000/01254859074079537426/*/1lHtMfINNtwfpTMQ2zJny_AiyFpHP9puM [following]
--2021-05-10 15:19:09--  https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n69

In [ ]:
# IMDE image 人臉資料集來源: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os 
import numpy as np
from tqdm import tqdm
import glob
import random

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout,
                    Activation, Flatten, Conv2D,GlobalAveragePooling2D,
                    MaxPooling2D)
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
wiki = pd.read_csv('/content/datasets/wiki_fillgdr.csv')
wiki_del = pd.read_csv('/content/datasets/del.csv')
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44291 entries, 0 to 44290
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          44291 non-null  int64  
 1   photo_takens  44291 non-null  int64  
 2   path          44291 non-null  object 
 3   gender        44291 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
wiki = wiki.drop(axis=0,index=wiki_del['0'].to_list()).reset_index(drop=True)
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44186 entries, 0 to 44185
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          44186 non-null  int64  
 1   photo_takens  44186 non-null  int64  
 2   path          44186 non-null  object 
 3   gender        44186 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
wiki.loc[wiki['gender']==2].index

Int64Index([12268, 15830, 21485], dtype='int64')

In [ ]:
wiki = wiki.drop(axis=0,index=wiki.loc[wiki['gender']==2].index).reset_index(drop=True)
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44183 entries, 0 to 44182
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          44183 non-null  int64  
 1   photo_takens  44183 non-null  int64  
 2   path          44183 non-null  object 
 3   gender        44183 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
# wiki.to_csv('/content/datasets/wiki_final.csv',index=False)

In [ ]:
def imagepath(index):
  index = index
  # print(wiki.iloc[index,:])
  path = os.path.join('/content/wiki_crop', wiki.iloc[index,2])
  return path

In [ ]:
def imgprocess(size,columns,sample_num):
  imgs = list()
  lbls = list()
  count = len(wiki[columns].unique())
  size = size
  for i in tqdm((sample_num), position=0, leave=True):
    img = cv2.imread(imagepath(i))
    img = cv2.resize(img, size)
    imgs.append(img)
    lbl = int(wiki[columns][i])
    lbl_onehot = np.zeros(count,dtype=float)
    lbl_onehot[lbl] = 1
    lbls.append(lbl_onehot)
  imgs = np.array(imgs)
  lbls = np.array(lbls)
  return {'images':imgs,'labels':lbls}

In [ ]:
# data index for random choosing
index_female = wiki[wiki['gender']==0].index
index_male = wiki[wiki['gender']==1].index
random_female = random.sample(list(index_female), 2000)
random_male = random.sample(list(index_male), 2000)
random_sample = random_female + random_male

In [ ]:
data = imgprocess((299,299),'gender',random_sample)

100%|██████████| 4000/4000 [00:07<00:00, 558.51it/s]


In [ ]:
data.keys()

dict_keys(['images', 'labels'])

In [ ]:
data['images'].shape

(4000, 299, 299, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['images'],data['labels'],test_size=0.2, random_state=500)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2, random_state=500)

In [ ]:
del data

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(2560, 299, 299, 3)
(2560, 2)
(800, 299, 299, 3)
(800, 2)
(640, 299, 299, 3)
(640, 2)


In [ ]:
num_class = 2
learning_rate = 0.001
loss = 'categorical_crossentropy'
metrics = 'accuracy'

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input

In [ ]:
model_InceptionV3 = InceptionV3(weights='imagenet', include_top=False,input_shape=(299,299,3))

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
x = GlobalAveragePooling2D()(model_InceptionV3.output)
x = Dense(64, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
new_model_InceptionV3 = Model(inputs=model_InceptionV3.input, outputs=predictions)

In [ ]:
new_model_InceptionV3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model_InceptionV3.trainable=False
new_model_InceptionV3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model_InceptionV3.trainable = False

trainable_layer = 100
for layer in model_InceptionV3.layers[-trainable_layer:]:
    layer.trainable = True

count = 0
for layer in new_model_InceptionV3.layers:
    print(layer, layer.trainable)
    count += 1
print(count)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f7cef088050> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f7cee6a6650> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f7cee6dc7d0> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f7cee62b110> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f7ce0494890> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f7ce0499f50> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f7ce04a4e10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f7ce04a1a10> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f7ce04a7e50> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f7ce049a850> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f7ce04aed90> False
<tensorflow.python.keras.layers.c

In [ ]:
optimizer = keras.optimizers.Adam(lr = learning_rate)
new_model_InceptionV3.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

In [ ]:
batch_size = 32
if len(x_train) % batch_size != 0:
  num_steps = len(x_train) // batch_size + 1
else:
  num_steps = len(x_train) // batch_size
num_epochs = 15

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  horizontal_flip=True,
                  fill_mode='wrap',
                  preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# !mkdir model-logs
filepath = './model-logs/inceptionV3_best_model.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
          filepath, monitor='val_loss', verbose=1, save_best_only=True,
          save_weights_only=False, mode='min', save_freq='epoch',
          options=None
          )
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, mode='min', verbose=1)

In [ ]:
history = new_model_InceptionV3.fit_generator(train_datagen.flow(x_train,y_train,batch_size=batch_size),
                        steps_per_epoch=num_steps,
                        epochs=num_epochs,
                        validation_data=test_datagen.flow(x_val,y_val,batch_size=batch_size),
                        callbacks = [early_stop,checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
80/80 [==============================] - 61s 697ms/step - loss: 0.2637 - accuracy: 0.8976 - val_loss: 0.8508 - val_accuracy: 0.8375

Epoch 00001: val_loss improved from inf to 0.85082, saving model to ./model-logs/inceptionV3_best_model.h5
Epoch 2/15
80/80 [==============================] - 55s 682ms/step - loss: 0.2088 - accuracy: 0.9129 - val_loss: 0.5618 - val_accuracy: 0.8375

Epoch 00002: val_loss improved from 0.85082 to 0.56180, saving model to ./model-logs/inceptionV3_best_model.h5
Epoch 3/15
80/80 [==============================] - 54s 672ms/step - loss: 0.1307 - accuracy: 0.9552 - val_loss: 0.4149 - val_accuracy: 0.8672

Epoch 00003: val_loss improved from 0.56180 to 0.41488, saving model to ./model-logs/inceptionV3_best_model.h5
Epoch 4/15
80/80 [==============================] - 54s 669ms/step - loss: 0.1280 - accuracy: 0.9504 - val_loss: 0.3889 - val_accuracy: 0.8734

Epoch 00004: val_loss improved from 0.41488 to 0.38886, saving model to ./model-logs/inceptionV

In [ ]:
# !mkdir model-old
!gdown -O /content/model-old/inceptionV3_241.h5  https://drive.google.com/uc?id=1sxas71-rtKfd3tovyYf1FdtHoL6IJ4Bq
!gdown -O /content/model-old/inceptionV3_216.h5 https://drive.google.com/uc?id=1P0oPF7J-3dpfmWWrIcTA-CKiaWy699kR
!gdown -O /content/model-old/inceptionV3_0.h5 https://drive.google.com/uc?id=150_X4rkaHUJdUPtlEiFIOMfIazFbyTiW
!gdown -O /content/model-old/inceptionV3_-47.h5 https://drive.google.com/uc?id=1rpNF7665obvwbA9pAdnx6pHHwNn-3CG4
!gdown -O /content/model-old/inceptionV3_-43.h5 https://drive.google.com/uc?id=1VXkaTvmRHLozLcndv7NIqjr2GTSoXQSo

Downloading...
From: https://drive.google.com/uc?id=1sxas71-rtKfd3tovyYf1FdtHoL6IJ4Bq
To: /content/model-old/inceptionV3_241.h5
199MB [00:01, 133MB/s]
Downloading...
From: https://drive.google.com/uc?id=1P0oPF7J-3dpfmWWrIcTA-CKiaWy699kR
To: /content/model-old/inceptionV3_216.h5
199MB [00:05, 37.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=150_X4rkaHUJdUPtlEiFIOMfIazFbyTiW
To: /content/model-old/inceptionV3_0.h5
140MB [00:03, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rpNF7665obvwbA9pAdnx6pHHwNn-3CG4
To: /content/model-old/inceptionV3_-47.h5
140MB [00:03, 37.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VXkaTvmRHLozLcndv7NIqjr2GTSoXQSo
To: /content/model-old/inceptionV3_-43.h5
140MB [00:03, 37.0MB/s]


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('./model-logs/inceptionV3_best_model.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 5s 116ms/step - loss: 0.3601 - accuracy: 0.8825


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('/content/model-old/inceptionV3_241.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 5s 116ms/step - loss: 0.7565 - accuracy: 0.8450


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('/content/model-old/inceptionV3_216.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 4s 116ms/step - loss: 0.7752 - accuracy: 0.8275


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('/content/model-old/inceptionV3_0.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 4s 116ms/step - loss: 0.9301 - accuracy: 0.8600


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('/content/model-old/inceptionV3_-47.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 4s 117ms/step - loss: 0.9554 - accuracy: 0.8500


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model('/content/model-old/inceptionV3_-43.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 4s 115ms/step - loss: 0.6813 - accuracy: 0.8587


In [ ]:
def to1Dtensor(y_input):
  y_list = []
  y_onehot = np.round(y_input)
  n_class = len(y_onehot[:][0])
  for j in y_onehot:
    for i in range(n_class):
      if j[i] == 1:
        y_list.append(i)
  return y_list

In [ ]:
predictions = test.predict(x=preprocess_input(x_test), verbose=1)

25/25 [==============================] - 4s 116ms/step


In [ ]:
y_test_1D = to1Dtensor(y_test)
pred_1D = to1Dtensor(predictions)

In [ ]:
# y_train_t = pd.DataFrame(y_train)
# y_test_t = pd.DataFrame(y_test)
# y_val_t = pd.DataFrame(y_val)
# display(y_train_t[y_train_t[0]==0].count())
# display(y_train_t[y_train_t[1]==0].count())
# display(y_test_t[y_test_t[0]==1].count())
# display(y_test_t[y_test_t[1]==1].count())
# display(y_val_t[y_val_t[0]==0].count())
# display(y_val_t[y_val_t[1]==0].count())

In [ ]:
# !mkdir imgs_test
# from PIL import Image

# for i in range(10):
#   img = Image.fromarray(x_val[i], 'RGB')
#   img.save(f'./imgs_test/my{i}{y_val[i]}.png')
#   img.show()

In [ ]:
tf.math.confusion_matrix(
            y_test_1D, pred_1D, num_classes=2, weights=None, dtype=tf.dtypes.int32,
            name=None
            )

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[292,  94],
       [ 19, 395]], dtype=int32)>